In [5]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
booking_json="""[
  {
    "bookingId": "BKG001",
    "seats": [
      {
        "SeatNum": "14B",
        "Time": "2025-11-10T09:15:00Z"
      },
      {
        "SeatNum": "3A",
        "Time": "2025-11-12T14:30:00Z"
      }
    ]
  },
  {
    "bookingId": "BKG002",
    "seats": [
      {
        "SeatNum": "22C",
        "Time": "2025-11-11T11:45:00Z"
      }
    ]
  },
  {
    "bookingId": "BKG003",
    "seats": [
      {
        "SeatNum": "9F",
        "Time": "2025-11-09T07:20:00Z"
      },
      {
        "SeatNum": "1D",
        "Time": "2025-11-10T10:00:00Z"
      }
    ]
  },
  {
    "bookingId": "BKG004",
    "seats": [
      {
        "SeatNum": "17A",
        "Time": "2025-11-08T16:10:00Z"
      },
      {
        "SeatNum": "5C",
        "Time": "2025-11-09T08:25:00Z"
      }
    ]
  },
  {
    "bookingId": "BKG005",
    "seats": [
      {
        "SeatNum": "11E",
        "Time": "2025-11-12T13:50:00Z"
      },
      {
        "SeatNum": "2B",
        "Time": "2025-11-13T09:40:00Z"
      }
    ]
  }
]
"""

StatementMeta(, c5f360d0-47a1-42d0-9cde-a0baf1a4f9bd, 7, Finished, Available, Finished)

In [28]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
import json

# Load JSON into DataFrame
data = json.loads(booking_json)
df = spark.createDataFrame(data)

# Explode the seats array
df_exp = df.withColumn("seat", F.explode("seats"))

# Select flattened structure
df_flat = df_exp.select(
    "bookingId",
    F.col("seat.SeatNum").alias("SeatNum"),
    F.col("seat.Time").alias("SeatTime")
)

# Convert SeatTime to timestamp for comparison
df_flat = df_flat.withColumn("SeatTime", F.to_timestamp("SeatTime"))

# Get the latest seat per bookingId
df_latest = (
    df_flat
    .withColumn(
        "rn", 
        F.row_number().over(
            Window.partitionBy("bookingId").orderBy(F.col("SeatTime").desc())
        )
    )
    .filter("rn = 1")
    .drop("rn")
)

df_latest.show(truncate=False)


StatementMeta(, c5f360d0-47a1-42d0-9cde-a0baf1a4f9bd, 30, Finished, Available, Finished)

+---------+-------+-------------------+
|bookingId|SeatNum|SeatTime           |
+---------+-------+-------------------+
|BKG001   |3A     |2025-11-12 14:30:00|
|BKG002   |22C    |2025-11-11 11:45:00|
|BKG003   |1D     |2025-11-10 10:00:00|
|BKG004   |5C     |2025-11-09 08:25:00|
|BKG005   |2B     |2025-11-13 09:40:00|
+---------+-------+-------------------+

